In [1]:
import logging
import os

import yaml
from igibson.scenes.igibson_indoor_scene import InteractiveIndoorScene
import igibson
import numpy as np
import pybullet as p
from igibson.utils.ig_logging import IGLogReader
from igibson import object_states
from igibson.objects.articulated_object import URDFObject
from igibson.scenes.empty_scene import EmptyScene
from igibson.simulator import Simulator
from igibson.utils.assets_utils import get_ig_model_path
from igibson.utils.utils import restoreState
from igibson.transition_model.utils.utils import convert_windows_to_linux_path
from igibson.utils.utils import parse_config
from igibson.utils.checkpoint_utils import save_internal_states

INFO:root:Importing iGibson (igibson module)
INFO:root:Assets path: D:\GitHub\behavior-vllm-eval\igibson\data/assets
INFO:root:Gibson Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/g_dataset
INFO:root:iG Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset
INFO:root:3D-FRONT Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/threedfront_dataset
INFO:root:CubiCasa5K Dataset path: D:\GitHub\behavior-vllm-eval\igibson\data/cubicasa_dataset
INFO:root:iGibson Key path: D:\GitHub\behavior-vllm-eval\igibson\data/igibson.key
INFO:root:Example path: D:\GitHub\behavior-vllm-eval\igibson\examples
INFO:root:Example config path: D:\GitHub\behavior-vllm-eval\igibson\examples\configs


torch is not available, falling back to rendering to memory(instead of tensor)


In [2]:
simulator=Simulator()

INFO:root:Loading D:\GitHub\behavior-vllm-eval\igibson\data/assets\models/mjcf_primitives/cube.obj
INFO:root:Loading D:\GitHub\behavior-vllm-eval\igibson\data/assets\models/mjcf_primitives/sphere8.obj


In [3]:
demo_path="D:\GitHub\behavior-vllm-eval\bottling_fruit_0_Benevolence_1_int_0_2024-04-09_20-24-57_replay.hdf5"
demo_path=convert_windows_to_linux_path(demo_path)

In [4]:
task = IGLogReader.read_metadata_attr(demo_path, "/metadata/atus_activity")
task_id = IGLogReader.read_metadata_attr(demo_path, "/metadata/activity_definition")
scene_id = IGLogReader.read_metadata_attr(demo_path, "/metadata/scene_id")

config_file = os.path.join(igibson.example_config_path, "behavior_segmentation_replay.yaml")
with open(config_file, "r") as f:
    config = yaml.safe_load(f)


config["task"] = task
config["task_id"] = task_id
config["scene_id"] = scene_id


In [5]:
config=parse_config(config)

In [14]:
for k,v in config.items():
    print(k,v)

scene_id Benevolence_1_int
clutter False
build_graph True
load_texture True
pybullet_load_texture True
should_open_all_doors True
debug False
texture_randomization_freq None
object_randomization_freq None
robot BehaviorRobot
task bottling_fruit
task_id 0
online_sampling False
target_dist_min 1.0
target_dist_max 10.0
goal_format polar
task_obs_dim 4
reward_type l2
success_reward 10.0
potential_reward_weight 1.0
collision_reward_weight -0.1
discount_factor 0.99
dist_tol 0.36
max_step 100
max_collisions_allowed 500
initial_pos_z_offset 0.1
collision_ignore_link_a_ids [0, 1, 2]
output ['proprioception', 'task_obs']
fisheye False
image_width 512
image_height 512
vertical_fov 90
depth_low 0.35
depth_high 3.0
n_horizontal_rays 220
n_vertical_beams 1
laser_linear_range 25.0
laser_angular_range 220.0
min_laser_dist 0.05
laser_link_name laser_link
depth_noise_rate 0.0
scan_noise_rate 0.0
visual_object_at_initial_target_pos True
target_visual_object_visible_to_agent False
hand_threshold 0.4
hand_

In [7]:
first_n = config.get("_set_first_n_objects", -1)
print("first_n",first_n)

first_n -1


In [8]:
scene = InteractiveIndoorScene(
    config["scene_id"],
    waypoint_resolution=config.get("waypoint_resolution", 0.2),
    num_waypoints=config.get("num_waypoints", 10),
    build_graph=config.get("build_graph", False),
    trav_map_resolution=config.get("trav_map_resolution", 0.1),
    trav_map_erosion=config.get("trav_map_erosion", 2),
    trav_map_type=config.get("trav_map_type", "with_obj"),
    pybullet_load_texture=config.get("pybullet_load_texture", False),
    texture_randomization=config.get("texture_randomization_freq", None) is not None,
    object_randomization=config.get("object_randomization_freq", None) is not None,
    object_randomization_idx=0,
    should_open_all_doors=config.get("should_open_all_doors", False),
    load_object_categories=config.get("load_object_categories", None),
    load_room_types=config.get("load_room_types", None),
    load_room_instances=config.get("load_room_instances", None),
)
if first_n != -1:
    scene._set_first_n_objects(first_n)
simulator.import_ig_scene(scene)

INFO:root:IndoorScene model: Benevolence_1_int
INFO:root:StaticIndoorScene scene: Benevolence_1_int
INFO:root:Category walls
INFO:root:Loading the following URDF template D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\urdf\Benevolence_1_int_walls.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scene_instances\20240410-170028_14413854528976483778_38552\walls_0.urdf
INFO:root:Category floors
INFO:root:Loading the following URDF template D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scenes\Benevolence_1_int\urdf\Benevolence_1_int_floors.urdf
INFO:root:Scale: [1. 1. 1.]
INFO:root:Number of splits: 1
INFO:root:Instantiating scene into the following urdfs:
INFO:root:D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scene_instances\20240410-170028_14413854528976483778_38552\floors_0.urdf
INFO:root:Category ceilings
INFO:

SCENE NAME: Benevolence_1_int


INFO:root:Instantiating scene into the following urdfs:
INFO:root:D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\scene_instances\20240410-170028_14413854528976483778_38552\console_table_9_0.urdf
INFO:root:Category sofa
INFO:root:Loading the following URDF template D:\GitHub\behavior-vllm-eval\igibson\data/ig_dataset\objects\sofa\875c7185cce6b8b0813204830fbed813\875c7185cce6b8b0813204830fbed813.urdf
INFO:root:Scale: [3.47685564 4.24746417 3.78256328]
INFO:root:Child of floating: sofa_10_pillow1
INFO:root:All children of the floating joint: sofa_10_pillow1
INFO:root:Child of floating: sofa_10_pillow2
INFO:root:All children of the floating joint: sofa_10_pillow2
INFO:root:Child of floating: sofa_10_pillow3
INFO:root:All children of the floating joint: sofa_10_pillow3
INFO:root:Child of floating: sofa_10_pillow4
INFO:root:All children of the floating joint: sofa_10_pillow4
INFO:root:Child of floating: sofa_10_pillow5
INFO:root:All children of the floating joint: sofa_10_pillow5
INFO:

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66]

In [9]:
for name, obj in simulator.scene.objects_by_name.items():
    print(name, obj)

walls <igibson.objects.articulated_object.URDFObject object at 0x000002256E21FFA0>
floors <igibson.objects.articulated_object.URDFObject object at 0x000002256E21FB80>
ceilings <igibson.objects.articulated_object.URDFObject object at 0x000002250E038D30>
shelf_0 <igibson.objects.articulated_object.URDFObject object at 0x000002250E5C4BB0>
breakfast_table_1 <igibson.objects.articulated_object.URDFObject object at 0x000002250E038F10>
straight_chair_2 <igibson.objects.articulated_object.URDFObject object at 0x000002250E5C4EE0>
straight_chair_3 <igibson.objects.articulated_object.URDFObject object at 0x000002250E5E5D00>
straight_chair_4 <igibson.objects.articulated_object.URDFObject object at 0x000002250E5E5100>
straight_chair_5 <igibson.objects.articulated_object.URDFObject object at 0x000002250E5D0C10>
shelf_6 <igibson.objects.articulated_object.URDFObject object at 0x000002250E602E80>
shelf_7 <igibson.objects.articulated_object.URDFObject object at 0x000002250E5FA820>
floor_lamp_8 <igibson

In [10]:
rst=save_internal_states(simulator)
print(rst)

{'objects': {'walls': {'InsideRoomTypes': None, 'VerticalAdjacency': None, 'HorizontalAdjacency': None, 'ContactBodies': None, 'AABB': None, 'Pose': None, 'InSameRoomAsRobot': None, 'InHandOfRobot': None, 'InReachOfRobot': None, 'InFOVOfRobot': None}, 'floors': {'InsideRoomTypes': None, 'VerticalAdjacency': None, 'HorizontalAdjacency': None, 'ContactBodies': None, 'AABB': None, 'Pose': None, 'Stained': {'particles': {'dirt_dump': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'random_bbox_dims': [(0.06978303312753696, 0.06978303312753696, 0.004), (0.03210554842610464, 0.03210554842610464, 0.004), (0.056927962124374384, 0.056927962124374384, 0.004), (0.06199925746225045, 0.06199925746225045, 0.004), (0.0619803048556766, 0.0619803048556766, 0.004), (0.07409790108150192, 0.07409790108150192, 0.004), (0.028720230033903765, 0.028720230033903765, 0.004), (0.08325066768239071, 0.08325066768239071, 0.004), (0.0545417671

In [17]:
for name, obj in simulator.scene.objects_by_name.items():
    print(name,';',obj)
    break

walls ; <igibson.objects.articulated_object.URDFObject object at 0x000002256E21FFA0>


In [19]:
for item in rst['objects'].items():
    print(item)

('walls', {'InsideRoomTypes': None, 'VerticalAdjacency': None, 'HorizontalAdjacency': None, 'ContactBodies': None, 'AABB': None, 'Pose': None, 'InSameRoomAsRobot': None, 'InHandOfRobot': None, 'InReachOfRobot': None, 'InFOVOfRobot': None})
('floors', {'InsideRoomTypes': None, 'VerticalAdjacency': None, 'HorizontalAdjacency': None, 'ContactBodies': None, 'AABB': None, 'Pose': None, 'Stained': {'particles': {'dirt_dump': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'random_bbox_dims': [(0.06978303312753696, 0.06978303312753696, 0.004), (0.03210554842610464, 0.03210554842610464, 0.004), (0.056927962124374384, 0.056927962124374384, 0.004), (0.06199925746225045, 0.06199925746225045, 0.004), (0.0619803048556766, 0.0619803048556766, 0.004), (0.07409790108150192, 0.07409790108150192, 0.004), (0.028720230033903765, 0.028720230033903765, 0.004), (0.08325066768239071, 0.08325066768239071, 0.004), (0.05454176718774663, 0.

In [12]:
for item in rst['objects']['window_53'].items():
    print(item)

('InsideRoomTypes', None)
('VerticalAdjacency', None)
('HorizontalAdjacency', None)
('ContactBodies', None)
('AABB', None)
('Pose', None)
('Stained', {'particles': {'dirt_dump': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'random_bbox_dims': [(0.08025553564202884, 0.08025553564202884, 0.004), (0.07150370144682901, 0.07150370144682901, 0.004), (0.03711059895644753, 0.03711059895644753, 0.004), (0.03476910343260062, 0.03476910343260062, 0.004), (0.09972689719050118, 0.09972689719050118, 0.004), (0.06172799664623922, 0.06172799664623922, 0.004), (0.05731734329351579, 0.05731734329351579, 0.004), (0.07964724251527555, 0.07964724251527555, 0.004), (0.05585092641497055, 0.05585092641497055, 0.004), (0.07460948637593952, 0.07460948637593952, 0.004), (0.05433029984624456, 0.05433029984624456, 0.004), (0.04234984612059626, 0.04234984612059626, 0.004), (0.08787421397133498, 0.08787421397133498, 0.004), (0.0663786070145